In [1]:

from glob import glob
import re

cc_on_pwc		  full_on_pwc	remover.ipynb
CSRNet			  gram.py	results
data			  __init__.py	results.ipynb
datasets		  labeler	run_spreadsheet.py
DDFlow			  LOI.py	testing.txt
DDFlow_pytorch		  LOI.pyc	thesis-loi-288313-328c2d373141.json
DRNet			  main.py	utils.py
DRNet_pytorch		  multi_run.sh	utils.pyc
finishing_spreadsheet.py  preprocess	v2_compare.ipynb
FlowNetPytorch		  __pycache__	video-to-traffic-flow-information


In [2]:
from datasets import basic_entities

In [3]:
def compile_to_frame_obj(dictio, load_labels=True):
    frame = basic_entities.BasicFrame(dictio['png'])
    if load_labels == True:
        with open(dictio['csv']) as f:
            lines = [line.rstrip() for line in f]
            
        if len(lines) > 1:
            for line in lines[1:]:
                coords = line.split(',')
                frame.add_point((int(coords[0]), int(coords[1])))
    return frame

In [4]:
def load_videos(base_path, load_labels=True):
    png_regex = re.compile('(.*)\/([0-9]+)-([0-9]{10}).([0-9]+).png\Z')
    csv_regex = re.compile('(.*)\/([0-9]+)-([0-9]{10})([0-9]+)-tags.csv\Z')
    frame_dict = {}
    for file in glob('{}*'.format(base_path)):
        png_match = png_regex.match(file)
        csv_match = csv_regex.match(file)
        if csv_match is not None:
            frame_number = int(csv_match.group(2))
            uid = int(csv_match.group(3))
        elif png_match is not None:
            frame_number = int(png_match.group(2))
            uid = int(png_match.group(3))
        else:
            print("Nothing!!! Something went wrong!!")
            print(file)
            break

        if frame_number not in frame_dict:
            frame_dict[frame_number] = {}

        if uid not in frame_dict[frame_number]:
            frame_dict[frame_number][uid] = {}

        if csv_match is not None:
            frame_dict[frame_number][uid]['csv'] = file
        elif png_match is not None:
            frame_dict[frame_number][uid]['png'] = file


    frame_numbers = sorted(list(frame_dict.keys()))
    videos = []
    for begin_frame in frame_dict[0]:
        video = basic_entities.BasicVideo(
            png_regex.match(frame_dict[0][begin_frame]['png']).group(1),
            load_labels
        )

        for frame_num in frame_numbers:
            added = False
            for i in range(-20, 2):
                if begin_frame+frame_num+i in frame_dict[frame_num]:
                    added = True
                    video.add_frame(compile_to_frame_obj(
                        frame_dict[frame_num][begin_frame+frame_num+i],
                        load_labels
                    ))
                    break

            if added == False:
                break
        videos.append(video)
        
    return videos
print(load_videos('data/AmsterdamPeds/'))

[<datasets.basic_entities.BasicVideo object at 0x7feaa85c6970>, <datasets.basic_entities.BasicVideo object at 0x7fea9a108280>, <datasets.basic_entities.BasicVideo object at 0x7fea9830ef40>]


In [18]:

import numpy as np
import math

dot1 = [0,0]
dot2 = [100, 200]
regions = 6
d_width = 20
d_height = 30


height_region = d_width
region_lines = []
line_points = np.linspace(np.array(dot1), np.array(dot2), num=regions+1).astype(int)
for i, point in enumerate(line_points):
    if i + 1 >= len(line_points):
        break

    point2 = line_points[i + 1]
    region_lines.append((tuple(list(point)),  tuple(list(point2))))
region_lines.reverse()

print(region_lines)
print(len(region_lines))

[((83, 166), (100, 200)), ((66, 133), (83, 166)), ((50, 100), (66, 133)), ((33, 66), (50, 100)), ((16, 33), (33, 66)), ((0, 0), (16, 33))]
6


In [27]:
height_region = d_width
width_region = d_height

line_length = math.sqrt(math.pow(dot1[0]-dot2[0], 2)+math.pow(dot1[1]-dot2[1], 2))
a_regions = math.floor(line_length / width_region)
side_perc = (1.0 - a_regions*width_region/line_length) / 2

inner_points = list(np.linspace(side_perc, 1.-side_perc, num=a_regions+1).astype(float))
inner_points.insert(0, 0.0)
inner_points.append(1.0)
line_points = []
for point_multi in inner_points:
    line_points.append((np.array(dot1) + point_multi*(np.array(dot2) - np.array(dot1))))
        
region_lines = []
for i, point in enumerate(line_points):
    if i + 1 >= len(line_points):
        break

    point2 = line_points[i + 1]
    region_lines.append((tuple(list(point)),  tuple(list(point2))))

region_lines.reverse()
print(region_lines)
print(len(region_lines))

[((96.95742752749558, 193.91485505499116), (100.0, 200.0)), ((83.54101966249685, 167.0820393249937), (96.95742752749558, 193.91485505499116)), ((70.12461179749812, 140.24922359499624), (83.54101966249685, 167.0820393249937)), ((56.70820393249938, 113.41640786499876), (70.12461179749812, 140.24922359499624)), ((43.291796067500634, 86.58359213500127), (56.70820393249938, 113.41640786499876)), ((29.875388202501895, 59.75077640500379), (43.291796067500634, 86.58359213500127)), ((16.458980337503156, 32.91796067500631), (29.875388202501895, 59.75077640500379)), ((3.0425724725044176, 6.085144945008835), (16.458980337503156, 32.91796067500631)), ((0.0, 0.0), (3.0425724725044176, 6.085144945008835))]
9


In [30]:
from scipy.spatial import distance
distance.euclidean(np.array(region_lines[1][0]), np.array(region_lines[1][1]))

29.999999999999986

array([[ 96.95742753, 193.91485505],
       [100.        , 200.        ]])